In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from data_apps_aws.sql import get_db_engine, get_db_data
from data_apps_aws.utils import make_outside_legend
from data_apps_aws.sql import *

from itables import show
import itables.options as opt
opt.maxBytes = 0

In [ ]:
plt.rcParams['figure.figsize'] = 14, 6
sns.set()

In [ ]:
db_con = get_db_engine('bfv_data')

Noteworthy data aspects:
- trainer: first / last name seems to be mixed up --> almost none are found, names are misclassified
- SV Bernried II exists twice 

## Data pipelines

### Overview selected leagues

In [ ]:
league_info = pd.read_csv('../data_pipes/bfv/bfv_league_links.csv')

n_leagues = league_info.shape[0]
print(f'Overall number of leagues: {n_leagues}')

In [ ]:
league_ids = []
for this_league_link in league_info['Link']:
    league_ids.append(this_league_link.split('https://www.bfv.de/wettbewerbe/meisterschaften/')[1])
    
league_info['league_id'] = league_ids

In [ ]:
n_leagues = league_info.groupby(['Saison', 'Info', 'Gebiet', 'Liga'])['Link'].count().to_frame()
n_leagues.rename({'Link': 'n_count'}, axis=1)

### League table scanning

In [ ]:
query = """
SELECT * FROM bfv.page_scan_logging;
"""

data_pipe_logging = get_db_data(query, db_con)

For each given league we extracted associated final tables.

In [ ]:
data_pipe_logging.query('job_type == "league_tables"').head(4)

For some leagues it did not work:

In [ ]:
not_successful_table_scans = data_pipe_logging.query('job_type == "league_tables" and success == 0')
not_successful_table_scans

In [ ]:
skipped_table_scans = data_pipe_logging.query('job_type == "league_tables" and success == 1 and skipped == 1')
assert skipped_table_scans.shape[0] == 0, 'We expect no skipped table scan'
skipped_table_scans

In [ ]:
league_info.set_index('league_id').loc[not_successful_table_scans['league_id'].values, :]

### Match-day scanning

For all leagues we also tried to find all games for two match days:

In [ ]:
not_successful_matchday_scans = data_pipe_logging.query('job_type == "matchday_games" and success == 0')
assert not_successful_matchday_scans.shape[0] == 1, 'We only know about a single failure so far. Code needs to be adapted otherwise'

not_successful_matchday_scans

In [ ]:
skipped_table_scans = data_pipe_logging.query('job_type == "matchday_games" and success == 1 and skipped == 1')
assert skipped_table_scans.shape[0] == 0, 'We expect no skipped table scan'
skipped_table_scans

In [ ]:
this_league_id, this_match_day = not_successful_matchday_scans['league_id'].values[0].split('#')

print(f'We did not get matchday games for matchday {this_match_day} of the following league:')

league_info.set_index('league_id').loc[this_league_id, :].to_frame()

### Match participant scanning

For all matches we tried to get all involved match participants:

In [ ]:
not_successful_match_scans = data_pipe_logging.query('job_type == "match_participants" and success == 0').copy()

all_match_links = []
for this_league_id in not_successful_match_scans['league_id']:
    all_match_links.append('https://www.bfv.de/spiele/' + this_league_id)
    
not_successful_match_scans['match_link'] = all_match_links

n_failed_match_scans = not_successful_match_scans.shape[0]
print(f'Number of failed match scans where match participants could not be found: {n_failed_match_scans}')

In [ ]:
for this_entry in not_successful_match_scans['match_link']:
    print(this_entry)

In [ ]:
not_successful_match_scans = data_pipe_logging.query('job_type == "match_participants" and success == 1 and skipped == 1')
# assert not_successful_matchday_scans.shape[0] == 1, 'We only know about a single failure so far. Code needs to be adapted otherwise'

not_successful_match_scans

## Data availability

Most players participate in more than 1 game. Some of them even participate in more than 2 games, even though we by construction targeted only 2 games per team. Reasons for this are:

- player is part of first and second team of a club, hence has multiple matches per match day. This could mean multiple matches per day, but actually match days of different leagues do not need to be synchronized with regards to calendar days. Hence, e.g. the 9th match day of the first and the second team could also have been taken place on different dates

- some team had multiple games listed on a single match day page. This can happen whenever some matches have been postponed


In [ ]:
query = """
SELECT *
FROM match_participants
"""

match_participants_raw = get_db_data(query, db_con)

# remove trainer
match_participants_raw = match_participants_raw[match_participants_raw['type'] != 'trainer']

In [ ]:
match_participants_raw['type'].unique()

In [ ]:
n_matches_per_player = match_participants_raw.groupby(['person_id'])['match_id'].nunique().sort_values()

Distribution of number of games per player:

In [ ]:
n_matches_per_player.hist()
plt.xlabel('Number of matches per player')
plt.ylabel('Number of players')

plt.show()

Players with highest number of matches:

In [ ]:
these_players = n_matches_per_player.tail(10).index.values

multiple_match_players = match_participants_raw.query('person_id in @these_players').groupby(['person_id', 'player_name', 'team_id', 'team'])['match_id'].nunique()
multiple_match_players = multiple_match_players.to_frame().reset_index().drop(columns=['person_id', 'team_id'])
multiple_match_players.rename({'match_id': 'n_matches'}, axis=1, inplace=True)
multiple_match_players.set_index('player_name')

Some teams have more than 2 matches in the sample. This can happen due to matches that have been postponed:

In [ ]:
n_matches_per_team = match_participants_raw.groupby(['team', 'team_id'])['match_id'].nunique().to_frame()
n_matches_per_team = n_matches_per_team.reset_index()
n_matches_per_team.rename({'match_id': 'n_matches'}, axis=1, inplace=True)
n_matches_per_team = n_matches_per_team.sort_values('n_matches')
n_matches_per_team.drop(columns=['team_id']).tail(10)

We can also look at the number of participants per match for each team:

In [ ]:
n_players_per_match_and_team = match_participants_raw.groupby(['team', 'team_id', 'match_id'])['person_id'].nunique().to_frame()
n_players_per_match_and_team = n_players_per_match_and_team.reset_index()
n_players_per_match_and_team.rename({'person_id': 'n_persons'}, axis=1, inplace=True)
n_players_per_match_and_team = n_players_per_match_and_team.sort_values('n_persons')
n_players_per_match_and_team.drop(columns=['team_id']).tail(10)

In [ ]:
n_players_per_match_and_team.sort_values('n_persons')

In case that some team was withdrawn from the championship, teams involved in one of the "cancelled" matches might have zeros players listed.

In [ ]:
n_bins = n_players_per_match_and_team['n_persons'].max() + 1 # because 0 could also occur
n_players_per_match_and_team.loc[:, ['match_id', 'n_persons']].set_index('match_id').hist(bins=n_bins)
plt.xlabel('Number of persons per team per match')
plt.show()

Create a look-up for match-ids and their respective championship:

In [ ]:
query = """
SELECT *
FROM match_day_links
"""

match_day_links_raw = get_db_data(query, db_con)

# remove entries without link
championship_match_affil = match_day_links_raw.loc[~match_day_links_raw['link'].isna()].copy()

In [ ]:
all_match_ids = []
for this_link in championship_match_affil['link'].values:
    all_match_ids.append(this_link.split('https://www.bfv.de/spiele/')[1])
championship_match_affil['match_id'] = all_match_ids
championship_match_affil.tail(3)

Show an example of a team with more matches than expected:

In [ ]:
# get team with most matches
this_team_id = n_matches_per_team['team_id'].tail(1).squeeze()
this_team_name = n_matches_per_team['team'].tail(1).squeeze()
this_team_match_ids = n_players_per_match_and_team.query('team_id == @this_team_id')['match_id'].values

this_team_matches_and_match_days = championship_match_affil.query('match_id in @this_team_match_ids').copy()
this_team_matches_and_match_days.insert(0, 'team_id', this_team_id)
this_team_matches_and_match_days.insert(0, 'team', this_team_name)
this_team_matches_and_match_days.drop(columns=['last_updated', 'match_id'], inplace=True)
this_team_matches_and_match_days.merge(league_info, left_on='championship_id', right_on='league_id')

## First name distributions

### Anonymous players

For some match participants we do not get a player name nor a person_id. We will throw out these entries.

In [ ]:
match_participants_raw[match_participants_raw['person_id'].isna()].head(2)

In [ ]:
anonymous_players_per_match = match_participants_raw[match_participants_raw['person_id'].isna()]
anonymous_players_per_match = anonymous_players_per_match.groupby(['team', 'team_id', 'match_id'])['first_name'].count().to_frame()
anonymous_players_per_match.rename({'first_name': 'n_anonymous'}, inplace=True, axis=1)
anonymous_players_per_match.reset_index().drop(columns=['team_id']).sort_values('n_anonymous').tail(8)

In [ ]:
n_overall_anonymous = anonymous_players_per_match.sum().squeeze()
print(f'Overall occurrence of anonymous players: {n_overall_anonymous}')

### Ambiguous names

Some names are not camel case and some are abbreviated:

In [ ]:
match_participants_ids = match_participants_raw[~match_participants_raw['person_id'].isna()]
match_participants_ids.head(3)

In [ ]:
names_per_person_id = match_participants_ids.loc[:, ['person_id', 'player_name', 'first_name']].groupby(['person_id']).nunique()

xx_msk = (names_per_person_id['player_name'] > 1) | (names_per_person_id['first_name'] > 1)
ambiguous_name_entries = names_per_person_id.loc[xx_msk, :]

ambiguous_name_ids = ambiguous_name_entries.index.values

ambiguous_name_entries.sort_values(['first_name', 'player_name']).tail(10)

In [ ]:
ambiguous_names = match_participants_ids[match_participants_ids['person_id'].isin(ambiguous_name_ids)].loc[:, ['person_id', 'player_name', 'first_name']]
ambiguous_names.sort_values('person_id').drop_duplicates()

### Abbreviations

In [ ]:
def name_abbrev(some_str):
    if "." in some_str:
        return True
    else:
        return False

In [ ]:
abbreviation_mapping = {'Alex.': 'Alexander',
                        'Johann.': 'Johannes',
                        'Reinh.': 'Reinhard',
                        'Christ.': 'Christian',
                        'Chris.': 'Christian',
                        'And.': 'Andreas',
                        'Thorst.': 'Thorsten',
                        'Juer.': 'Jürgen',
                        'Mich.': 'Michael',
                        'Joh.': 'Johannes',
                        'Seb.': 'Sebastian',
                        'Sebasti.': 'Sebastian',
                        'Sebast.': 'Sebastian',
                        'Domin.': 'Dominik',
                        'Rud.': 'Rudolph',
                        'Benj.': 'Benjamin',
                        'Friedr.': 'Friedrich',
                        'Christo.': 'Christoph',
                        'Matth.': 'Matthias',
                       }

In [ ]:
xx_msk = match_participants_ids['first_name'].apply(lambda x: name_abbrev(x))
abbr_names = match_participants_ids.loc[xx_msk, :]
abbr_names = abbr_names.loc[:, ['person_id', 'player_name', 'first_name', 'team']].drop_duplicates()
abbr_names['inferred'] = abbr_names['first_name'].map(abbreviation_mapping)
abbr_names

### Map to standardized names

First step is to find a single and standardized first name for each player:

In [ ]:
person_infos = match_participants_ids.loc[:, ['person_id', 'team', 'player_name', 'first_name', 'match_id']]

# deal with name abbreviations
person_infos['std_first_name'] = person_infos['first_name'].map(abbreviation_mapping)
xx_msk = person_infos['std_first_name'].isna()
person_infos.loc[xx_msk, 'std_first_name'] = person_infos.loc[xx_msk, 'first_name']

In [ ]:
n_players = person_infos['person_id'].nunique()

In [ ]:
# deal with ambiguous names
person_infos = person_infos.merge(championship_match_affil.loc[:, ['match_day', 'match_id']], how='left')

# get maximum match day per person id
max_match_day_per_person = person_infos.groupby(['person_id'])['match_day'].idxmax()
person_infos = person_infos.loc[max_match_day_per_person.values, :]

assert n_players == person_infos['person_id'].nunique(), 'Number of unique person-IDs did change'

person_infos = person_infos.loc[:, ['person_id', 'player_name', 'first_name', 'std_first_name']]

In [ ]:
# make lower case
person_infos['std_first_name'] = person_infos['std_first_name'].map(lambda x: x.lower())

## Name distribution

In [ ]:
name_occur = person_infos.groupby(['std_first_name'])['person_id'].nunique().to_frame()
name_occur.columns = ['n_count']
name_occur = name_occur.sort_values('n_count', ascending=False)
name_occur.head(5)

In [ ]:
name_occur.head(30).plot(kind='bar')
plt.title('Number of occurrences per first name')
plt.show()

## Names not found

In [ ]:
query = """
SELECT *
FROM match_participants
"""

match_participants_raw = get_db_data(query, db_con)

In [ ]:
match_participants_raw.head(4)

In [ ]:
query = """
SELECT *
FROM page_scan_logging_forebears
"""

name_scans = get_db_data(query, db_con)
name_scans.head(8)

In [ ]:
player_scan_results = match_participants_raw.loc[:, ['team', 'player_name', 'first_name', 'type', 'person_id']].copy()
player_scan_results.rename({'first_name': 'name'}, axis=1, inplace=True)
player_scan_results = player_scan_results.merge(name_scans.loc[:, ['name', 'success']], how='left')
player_scan_results.head(4)

In [ ]:
# throw out trainers
xx_msk = player_scan_results['type'] == 'trainer'
player_scan_results_no_trainer = player_scan_results.loc[~xx_msk, :]

team_scan_results = player_scan_results_no_trainer.drop(columns='type').groupby(['team', 'person_id'])['success'].sum().copy()
team_scan_results = team_scan_results.to_frame()
xx_msk = team_scan_results['success'] > 1
team_scan_results.loc[xx_msk, 'success'] = 1

team_scan_results = team_scan_results.reset_index()
team_scan_results = team_scan_results.groupby(['team', 'success']).count()
team_scan_results = team_scan_results.reset_index().pivot(index='team', columns='success', values='person_id').fillna(0)
team_scan_results['total'] = team_scan_results.sum(axis=1)
team_scan_results['name_info_exists'] = team_scan_results[1.0] / team_scan_results['total']
team_scan_results.sort_values('name_info_exists').head(10)

In [ ]:
player_scan_results_no_trainer.query('team == "FC Espanol"').drop_duplicates()

In [ ]:
player_scan_results.groupby(['team'])['person_id'].nunique().sort_values().to_frame().tail(10)

In [ ]:
name_scans['success'].value_counts()

In [ ]:
player_scan_results_no_trainer.loc[:, ['name', 'success']].drop_duplicates()['success'].value_counts()

In [ ]:
names_not_found = player_scan_results_no_trainer[player_scan_results_no_trainer['success'] == 0].loc[:, ['name', 'success']].drop_duplicates().head(20)
names_not_found.reset_index(inplace=True, drop=True)
names_not_found

In [ ]:
team_player_classifications = player_scan_results_no_trainer.drop_duplicates()
team_player_classifications = team_player_classifications.drop(columns='person_id')
team_player_classifications

In [ ]:
show(team_player_classifications)

## DEV

In [ ]:
query = """
SELECT *
FROM match_participants
"""

all_players = get_db_data(query, db_con)

In [ ]:
unique_players = all_players.loc[:, ['person_id', 'first_name']].drop_duplicates()

## Name distribution

In [ ]:
bfv_most_freq_players = unique_players['first_name'].value_counts().to_frame().reset_index()
bfv_most_freq_players.columns = ['first_name', 'sample_count']
bfv_most_freq_players['name'] = bfv_most_freq_players['first_name'].map(lambda x: x.lower())
bfv_most_freq_players.loc[:, ['name', 'sample_count']].set_index('name').head(40).plot(kind='bar')
plt.show()

## Name origin

In [ ]:
query = """
SELECT *
FROM forebears_name_lookup
"""

name_origin_data_raw = get_db_data(query, db_con)

In [ ]:
name_origin_data_raw['name'] = name_origin_data_raw['name'].map(lambda x: x.lower())

In [ ]:
name_origin_data_raw.groupby('name')['name_count'].sum().sort_values(ascending=False).head(30).to_frame().plot(kind='bar')
plt.title('Global name occurrences')
plt.show()

In [ ]:
most_freq_sample_occurs = name_origin_data_raw.query('country == "Germany"').loc[:, ['name', 'name_count']].sort_values('name_count', ascending=False).set_index('name')
most_freq_sample_occurs.head(20).plot(kind='bar')
plt.title('Most frequent occurrences in Germany')
plt.show()

In [ ]:
# compute German frequency per name

In [ ]:
german_occurs = name_origin_data_raw.query('country == "Germany"').loc[:, ['name', 'name_count']]
german_occurs = german_occurs.rename({'name_count': 'count_germany'}, axis=1)

global_occurs = name_origin_data_raw.loc[:, ['name', 'name_count']].groupby('name').sum()
global_occurs = global_occurs.reset_index()
global_occurs = global_occurs.rename({'name_count': 'count_global'}, axis=1)

german_occur_freq = global_occurs.merge(german_occurs, how='left').fillna(0)
german_occur_freq['german_freq'] = german_occur_freq['count_germany'] / german_occur_freq['count_global']
german_occur_freq = german_occur_freq.sort_values('german_freq', ascending=False)
german_occur_freq.head(30)

In [ ]:
german_occur_freq.set_index('name').loc[most_freq_sample_occurs.index, :].head(30)

In [ ]:
# pick most frequent country per name
max_idx = name_origin_data_raw.groupby('name')['name_count'].idxmax()
most_freq_occur_countries = name_origin_data_raw.loc[max_idx, ['name', 'country', 'name_count']]
most_freq_occur_countries.sort_values('name')
most_freq_occur_countries = most_freq_occur_countries.reset_index(drop=True)

In [ ]:
assert most_freq_occur_countries.shape[0] == len(name_origin_data_raw['name'].unique())

In [ ]:
german_occur_freq

In [ ]:
name_country_lookup = most_freq_occur_countries.merge(german_occur_freq).set_index('name')
name_country_lookup = name_country_lookup.reindex(bfv_most_freq_players['name'])
name_country_lookup.head(40)

In [ ]:
name_country_lookup.head(80).loc[:, 'german_freq'].plot(kind='bar')
plt.show()

In [ ]:
names_above_freq_threshold = name_country_lookup.dropna()
xx_msk = names_above_freq_threshold['german_freq'] > 0.1
names_above_freq_threshold.loc[xx_msk, :]

In [ ]:
classification_threshold = 0.05

country_classification = name_country_lookup.reset_index().loc[:, ['name', 'german_freq']].copy()
country_classification['country'] = 'unknown'

xx_msk = country_classification.loc[:, 'german_freq'] > classification_threshold
country_classification.loc[xx_msk, 'country'] = 'german'

xx_msk = country_classification.loc[:, 'german_freq'] <= classification_threshold
country_classification.loc[xx_msk, 'country'] = 'foreign'

country_classification.drop(columns=['german_freq'], inplace=True)

In [ ]:
mapped_players = unique_players.copy()
mapped_players['name'] = mapped_players['first_name'].map(lambda x: x.lower())
mapped_players = mapped_players.merge(country_classification, how='left')

In [ ]:
mapped_players['country'].value_counts()

## Per team

In [ ]:
mapped_players.head(3)

In [ ]:
mapped_players.shape

In [ ]:
all_players.head(2)

In [ ]:
all_players.merge(mapped_players)

In [ ]:
club_players = all_players.merge(mapped_players)

In [ ]:
club_players.loc[:, ['team', 'match_id', 'person_id']].groupby(['team']).nunique().sort_values('person_id')

In [ ]:
club_nations = club_players.loc[:, ['team', 'country', 'person_id']].groupby(['team', 'country'])['person_id'].nunique().to_frame().reset_index()
club_nations = club_nations.pivot(index='team', columns='country', values='person_id').fillna(0)
club_nations['total'] = club_nations.sum(axis=1)
club_nations['german_ratio'] = club_nations['german'] / club_nations['total']
club_nations = club_nations.sort_values('german_ratio')
club_nations.head(20)

In [ ]:
club_nations.tail(20)

In [ ]:
club_nations.drop(columns='german_ratio').sum()

In [ ]:
club_nations['german_ratio'].plot(kind='hist')
plt.xlabel('Ratio of german players per club')
plt.show()

In [ ]:
all_players.merge(mapped_players).loc[:, ['team', 'match_id', 'person_id', 'country']].groupby(['team']).nunique().sort_values('person_id')

In [ ]:
all_players.shape

In [ ]:
query = """
SELECT *
FROM fairness_tables
"""

fairness_raw = get_db_data(query, db_con)

In [ ]:
fairness_vs_migration = fairness_raw.loc[:, ['team', 'quote']].merge(club_nations.reset_index().loc[:, ['team', 'german_ratio']])

plt.scatter(fairness_vs_migration['german_ratio'], fairness_vs_migration['quote'])
plt.xlabel('Ratio of German players')
plt.ylabel('Fairness score (lower means more fair)')
plt.show()

## Dev

In [ ]:
name_country_lookup = unique_players.copy()
name_country_lookup['name'] = name_country_lookup['first_name'].map(lambda x: x.lower())
name_country_lookup = name_country_lookup.merge(most_freq_occur_countries)

In [ ]:
name_country_lookup.groupby(['name', 'country'])['person_id'].count().sort_values(ascending=False).head(40)

In [ ]:
germany_prob_lookup = unique_players.copy()

In [ ]:
name_country_lookup.query('country == "Germany"').

In [ ]:
germany_prob_lookup